# Dinaminiai duomenys

In [ ]:
import json
from zipfile import ZipFile
import os

import plotly.express as px
import pandas as pd

working_directory = f'{os.getcwd()}/../data/saltiniai/vintra/'
gtfs_files_directory = os.path.join(working_directory, 'gtfs')

mapbox_access_token = open("../.mapbox_token").read()
px.set_mapbox_access_token(mapbox_access_token)

lithuania_center = {'lat': 55.169438, 'lon': 23.881275}

with open('../data/geojson/municipalities.geojson', 'r') as municipalities_geojson_file:
    municipalities_geojson = json.load(municipalities_geojson_file)

## Duomenų teikimo privalomumas
Kokioms transporto rūšims turėtų būti teikiami transporto priemonių geografinių pozizijų duomenys į VINTRA? Pagal viešojo transporto kelionių duomenų kaupimo tvarkos aprašą šie duomenys turėtų turi apimti{cite}`sumin_vintra_duomenu_kaupimo_tvarka`:
- Vietinio (miesto ar priemiestinio) reguliarųjį susisiekimą;
- Tolimąjį reguliarųjį susisiekimą;
- Tarptautinį reguliarųjį susisiekimą;
- Keleivių vežimą geležinkeliais;

## Duomenys teikiami į VINTRA

Transporto priemonių pozicijų duomenis VINTRA periodiškai pasiima iš operatorių / vežėjų `.csv` formatu.

Toliau pateikiamas duomenų kontraktas, kurį turi atitikti transporto priemonių pozicijų duomenys:

| Atributas                 | Privalomumas                          | Aprašymas                                                                                                                                                                                                                                         |
|---------------------------|---------------------------------------|---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------|
| **Transportas**           | Privaloma                             | Tansporto priemonės, rūšis, pvz., „Autobusai“, „Troleibusai“.                                                                                                                                                                                     |
| **Marsrutas**             | Privaloma                             | Maršruto, kuriuo važiuoja transporto priemonė, numeris, pvz., „85“, „3g“.                                                                                                                                                                         |
| **Reisas**                | Privaloma (esant techninei galimybei) | Reiso, kurį vykdo transporto priemonė, identifikatorius, pvz., „2777540926“, „32001“, „3g-05_1020“. Identifikatorius *privalo* atititki GTFS atributą „trip_id“                                                                                   |
| **MasinosNumeris**        | Neprivaloma                           | Transporto priemonės identifikacinis (garažo) numeris, pvz., „252“, „771“.                                                                                                                                                                        |
| **Ilguma**                | Privaloma                             | Pateikiama transporto priemonės padėties ilguma WGS84 koordinačių sistemoje (DD.DDDDDD formatu), padauginta iš 1000000, pvz., „25285118“ , kas reiškia 25,285118 laipsnius.                                                                       |
| **Platuma**               | Privaloma                             | Pateikiama transporto priemonės padėties platuma WGS84 koordinačių sistemoje (DD.DDDDDD formatu), padauginta iš 1000000, pvz., „54671218“, kas reiškia 54,671218 laipsnius.                                                                       |
| **Greitis**               | Neprivaloma                           | Transporto priemonės judėjimo greitis (km/h), pvz., „9“, „17“.                                                                                                                                                                                    |
| **Azimutas**              | Privaloma (esant techninei galimybei) | Transporto priemonės judėjimo azimutas (laipsniais), pvz., „352“, „87“.                                                                                                                                                                           |
| **ReisoPradziaMinutemis** | Neprivaloma                           | Reiso, kurį vykdo transporto priemonė, pradžia minutėmis (skaičiuojant nuo vidurnakčio, tai yra 00:00 laiko), pvz., „696“, kas reiškia laiką „11:36“.                                                                                             |
| **NuokrypisSekundemis**   | Privaloma (esant techninei galimybei) | Transporto priemonės vėlavimas (sekundėmis), lyginant su planuotu reiso vykdymu (neigiamos reikšmės reiškia skubėjimą), pvz., „120“ reiškia, kad transporto priemonė vėluoja 2 minutes, „-90“ reiškia, kad transporto priemonė skuba 1,5 minutės. |

## Faktiškai teikiami duomenys į VINTRA
2022 balandžio 28 d. buvo faktiškai patikrinta, kokia dalis vežėjų teikia transporto priemonių geografinių pozicijų duomenis į VINTRA.

### Vietinis reguliarusis susisiekimas
| Savivaldybė         | Transporto priemonių pozicijų per parą |
|---------------------|----------------------------------------|
| Vilniaus m. sav.    | 572 736                                |
| Klaipėdos m. sav.   | 134 714                                |
| Šiaulių m. sav.     | 60 530                                 |
| Alytaus m. sav.     | 24 973                                 |
| Kretingos r. sav.   | 8 166                                  |
| Druskininkų sav.    | 2 995                                  |
| Rokiškio r. sav.    | 2 845                                  |
| Ukmergės r. sav.    | 2 152                                  |
| Biržų r. sav.       | 1 578                                  |
| Širvintų r. sav.    | 915                                    |
| Kazlų Rūdos sav.    | 497                                    |
| Marijampolės sav.   | 496                                    |
| Vilkaviškio r. sav. | 494                                    |

```{admonition} Savivaldybių transporto priemonių geografinės pozicijos duomenų trūkumas
:class: warning
14 Lietuvos savivaldybių yra teikiami transporto priemonių geografinės pozicijos duomenys. Likusių savivaldybių dinaminių duomenų VINTRA nėra.
```

In [ ]:
dynamic_df = pd.read_csv(os.path.join(working_directory, 'vintra-transporto-pozicijos-savivaldybes.csv'))

dynamic_df['Transporto pozicijų kiekis'] = dynamic_df['Įkeltos transporto pozicijos per dieną (balandžio 28)'].fillna('0')


fig_dynamic = px.choropleth(
    dynamic_df,
    geojson=municipalities_geojson,
    locations="Savivaldybė",
    featureidkey="properties.name",
    color='Statusas',
    fitbounds='locations',
    basemap_visible=False,
    projection="mercator",
    color_discrete_sequence=['#00CC96', '#FFA15A', '#EF553B'],
    category_orders={"Statusas": ["Teikiama", "Teikiama, tačiau nesurišta su reisu", "Neteikiama"]},
    hover_data=['Transporto pozicijų kiekis']

)

fig_dynamic.update_layout(
    margin={"r": 0, "l": 0, "b": 0},
    title_text='Vietinio susisiekimo transporto priemonių geografinės padėties duomenų teikimas (2022-04-28)',
)
fig_dynamic.show()

```{admonition} Transporto priemonių pozicijų duomenys nėra surišti su reisais
:class: warning
9 Lietuvos savivaldybių teikiami vietinio susisiekimo transporto priemonių geografinės pozicijos duomenys nėra surišti su reisais. Tai reiškia, kad tokie duomenys teikia mažiau naudos, nes nėra  galimybės nustatyti kiek laiko vėluoja transporto priemonė ar atlikti detalesnės duomenų analizės.
```

### Tolimasis susisiekimas
Tolimajame susisiekime tik viena bendrovė teikia transporto priemonių geografinių pozizijų duomenys į VINTRA (iš viso per parą 57 198 transporto priemonių geografinės padėties pozicijų).

Visi kiti vežėjai transporto priemonių geografinės padėties pozicijų duomenų į VINTRA neteikia.

```{admonition} Tolimojo susisiekimo vežėjų duomenų trūkumas
:class: warning
Nors pagal LTSA pasirašomas sutartis{cite}`ltsa_vezeju_prievole_teikti` ir viešojo transporto kelionių duomenų kaupimo tvarkos aprašą{cite}`sumin_vintra_duomenu_kaupimo_tvarka` tolimojo susisiekimo vežėjai privalo teikti transporto priemonių geografinės pozicijos duomenis į VINTRA, tačiau tą daro tik 1 iš 37 vežėjų.
```

## Stops.lt duomenys
Dalis Lietuvos savivaldybių naudoja Stops.lt sistemą iš kurios transporto priemonių geografinės padėties duomenys patenka į VINTRA. Duomenys yra teikiami VINTRA pritaikytu formatu. Toliau pateikiama lentelė su savivaldybėmis naudojančiomis Stops.lt sistema ir dinaminių duomenų šaltiniais.

### Prieš atnaujinimą

| Savivaldybė                                        | Duomenų nuoroda (iki 2022 balandžio 19 d.)                                               |
|----------------------------------------------------|------------------------------------------------------------------------------------------|
| [Vilniaus m. sav.](https://stops.lt/vilnius/)      | [https://www.stops.lt/vilnius/gps_full.txt](https://www.stops.lt/vilnius/gps_full.txt)   |
| [Kauno m. sav.](https://stops.lt/kaunas/)          | [https://www.stops.lt/kaunas/gps_full.txt](https://www.stops.lt/kaunas/gps_full.txt)     |
| [Klaipėdos m. sav.](https://stops.lt/klaipeda/)    | [https://www.stops.lt/klaipeda/gps_full.txt](https://www.stops.lt/klaipeda/gps_full.txt) |
| [Panevėžio m. sav](https://stops.lt/panevezys/)    | [https://www.stops.lt/panevezys/gps.txt](https://www.stops.lt/panevezys/gps.txt)         |
| [Alytaus m. sav.](https://stops.lt/alytus/)        | [https://www.stops.lt/alytus/gps.txt](https://www.stops.lt/alytus/gps.txt)               |
| [Druskininkų sav.](https://stops.lt/druskininkai/) | [https://www.stops.lt/druskininkai/gps.txt](https://www.stops.lt/druskininkai/gps.txt)   |

#### Integravimo į VINTRA problemos

```{admonition} Integralumo su statiniais duomenimis nebuvimas
:class: error
Pateikiamas reisas nesiriša su statiniais duomenimis ir pažeidžia VINTRA specifikaciją. Toks integralumo nebuvimas neleidžia sužinoti kokį reisą atitinka transporto priemonių geografinė pozicija. Problemai ištaisyti duomenų naudotojai turėdavo taikyti sudėtingus metodus pvz., trajektorijos analizę.
```

```{admonition} Skirtingų duomenų formatų pateikimas
:class: warning
Pateikiami miestų duomenys skiriasi savo formatu ir pateikia skirtinga informaciją (pvz., Vilniaus m. sav. pateikiamas `reisoID`, Kauno m. sav. ne, Panevėžio m. sav. duomenys grįžta be antraštės) dėl to yra pažeidžiama VINTRA specifikacija, o įmonėms norinčioms naudoti duomenis reikia skirtingai apdoroti kiekvienos savivaldybės duomenis.
```
#### Integravimo į kelionių planavimo programas problemos
Jei šie duomenys būtų integruojami tiesiogiai į kelionių planavimo programas tuomet būtų susiduriama su tokiomis problemomis.

```{admonition} Formato problemos
:class: warning
Realaus laiko transporto priemonių geografinės padėties duomenys pateikiami ne SIRI (pagal ITS direktyvą{cite}`directive_multimodal_travel_information_services_main`) ar GTFS Realtime (pasaulinis standartas) formatais.  Dėl šios priežasties nėra galimybės šių duomenų integruoti į jau egzistuojančias kelionių planavimo priemones pvz., Google Maps, Apple Maps ir kt.
```

```{admonition} Duomenų licenzijos nepateikimas
:class: warning
Nepateikiama duomenų naudojimo licenzija dėl šios priežasties nėra aišku, ką duomenų naudotojai gali ir ko negali daryti su duomenimis.
```

```{admonition} Nepateikiami metaduomenys
:class: warning
Nepateikiami metaduomenys dėl to duomenų naudotojas priverstas spėlioti kokia yra duomenų struktūra, kokios yra atribūtų reikšmės ir kokiais atvejais duomenų gali trūkti.
```



### Po atnaujinimo
Susisiekus su Stops.lt kūrėjais UAB „Merakas“ dėl integralumo su statiniais duomenimis nebuvimo (sąryšio per `Reisas`) problema buvo 2022 balandžio 19 d. buvo pašalinta bei buvo atnaujintos duomenų nuorodos. Taip pat buvo ištaisyta problema dėl skirtingų formatų naudojimo miestuose. Toliau pateikiama lentelė su atnaujintais duomenų šaltiniais.

| Savivaldybė                                        | Atnaujinta duomenų nuoroda                                                                             |
|----------------------------------------------------|--------------------------------------------------------------------------------------------------------|
| [Vilniaus m. sav.](https://stops.lt/vilnius/)      | [https://www.stops.lt/vilnius/gps_full_v2.txt](https://www.stops.lt/vilnius/gps_full_v2.txt)           |
| [Kauno m. sav.](https://stops.lt/kaunas/)          | [https://www.stops.lt/kaunas/gps_full_v2.txt](https://www.stops.lt/kaunas/gps_full_v2.txt)             |
| [Klaipėdos m. sav.](https://stops.lt/klaipeda/)    | [https://www.stops.lt/klaipeda/gps_full_v2.txt](https://www.stops.lt/klaipeda/gps_full_v2.txt)         |
| [Panevėžio m. sav](https://stops.lt/panevezys/)    | [https://www.stops.lt/panevezys/gps_full_v2.txt](https://www.stops.lt/panevezys/gps_full_v2.txt)       |
| [Alytaus m. sav.](https://stops.lt/alytus/)        | [https://www.stops.lt/alytus/gps_full_v2.txt](https://www.stops.lt/alytus/gps_full_v2.txt)             |
| [Druskininkų sav.](https://stops.lt/druskininkai/) | [https://www.stops.lt/druskininkai/gps_full_v2.txt](https://www.stops.lt/druskininkai/gps_full_v2.txt) |

Visgi, buvo pakeistas atributo pavadinimas iš `Reisas` į `ReisoID` dėl to atsirado papildomas VINTRA kontrakto neatitikimas.


## Apibendrinimas

Toliau pateikiama į VINTRA teikiamų duomenų suvestinė.

| Teikėjai                         | Transporto priemonių geografinės pozicijos |
|----------------------------------|--------------------------------------------|
| Savivaldybės                     | 14 savivaldybių                            |
| Tolimojo susisiekimo vežėjai     | 1 iš 37 vežėjų                             |
| Tarptautinio susisiekimo vežėjai | Neteikia                                   |
| Lietuvos geležinkeliai           | Neteikia                                   |

```{admonition} Tarptautinio susisiekimo vežėjų duomenų trūkumas
:class: warning
Nors pagal viešojo transporto kelionių duomenų kaupimo tvarkos aprašą{cite}`sumin_vintra_duomenu_kaupimo_tvarka` tarptautinio susisiekimo vežėjai turi teikti transporto priemonių geografinės pozicijos duomenis į VINTRA, tačiau to nedaro nė vienas vežėjas.
```

```{admonition} Lietuvos geležinkelių duomenų trūkumas
:class: warning
Nors pagal viešojo transporto kelionių duomenų kaupimo tvarkos aprašą{cite}`sumin_vintra_duomenu_kaupimo_tvarka` geležinkeliai turi teikti transporto priemonių geografinės pozicijos duomenis į VINTRA, tačiau duomenys nėra teikiami.
```